In [104]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [105]:
import sys
sys.path.append('/gdrive/My Drive/2nd Sem/IBM/Covid Project/Code/Scraping Twitter Data')

In [106]:
import private
import re
import tweepy
import difflib
import pandas as pd
import numpy as np
import pickle
import copy
import matplotlib.pyplot as plt


from datetime import date

In [107]:
auth = tweepy.OAuthHandler(private.CONSUMER_KEY, private.CONSUMER_SECRET)
auth.set_access_token(private.OAUTH_TOKEN, private.OAUTH_TOKEN_SECRET)
api = tweepy.API(auth)

#Extracting Hindi Covid Tweets

## Twitter Scraped Features Description


![alt text](https://drive.google.com/uc?id=1LAjmD-RIR-oswQ0ocKaXK708Z7thsrAs)









In [108]:
def get_list_of_user_mentions(tweet):
    user_handles = []

    try:
            
        mentions = tweet._json['entities']['user_mentions']
        for item in mentions:
            user_handles.append(item['screen_name'])
        return user_handles

    except:
        return user_handles

    

In [109]:
def get_list_of_photos(tweet):
    photos_list = []

    try:
            
        photos = tweet._json['entities']['media']
        for item in photos:
            photos_list.append(item['media_url_https'])
        return photos_list
    
    except:
        return photos_list

In [110]:
def get_replies_count(tweet,user_handle):
    
    try:
            
        replies=[] 
        q = 'to:'+user_handle
        for full_tweets in tweepy.Cursor(api.user_timeline,screen_name=user_handle,timeout=999999).items(10):
            for tweet in tweepy.Cursor(api.search,q, since_id=tweet_id, result_type='recent',tweet_mode='extended',timeout=999999).items(1000):
                if hasattr(tweet, 'in_reply_to_status_id_str'):
                    if (tweet.in_reply_to_status_id_str==full_tweets.id_str):
                        replies.append(tweet.full_text)

        #Removing Duplicate Replies
        replies = list(dict.fromkeys(replies))
        return len(replies)

    except:
        return 0

In [111]:
def get_all_hashtags(tweet):
    hashtags_list = []

    hashtags = tweet._json['entities']['hashtags']
    for item in hashtags:
        hashtags_list.append(item['text'])


    return hashtags_list

In [112]:
def get_retweet_id(tweet):
    try:
        retweet_id = tweet._json['retweeted_status']['id']
        return retweet_id
    except:
        return 0

In [113]:
def get_retweet_date(tweet):
    try:
        retweet_date = tweet._json['retweeted_status']['created_at']
        return retweet_date
    except:
        return ""

In [114]:
def get_retweet_user_id(tweet):
    try:
        retweet_user_id = tweet._json['retweeted_status']['user']['id']
        return retweet_user_id
    except:
        return ""

In [115]:
def check_tweet_exists(test_tweet, texts_list):
    # print("Is this function even running?")
    counter = 1
    test_tweet     = test_tweet.split()

    for existing_tweet in texts_list:
        existing_tweet = existing_tweet.split()
        n              = len(existing_tweet)
        m              = 0
        for word in test_tweet:
            if word in existing_tweet:
                m = m + 1
                if(m/n > 0.5):                                             ### If the new tweet is 50% similar with existing tweet then don't append it
                    #print("Helllo this tweet has been rejected") 
                    return True
        # print("This tweet has passed test number ",counter)
        counter = counter + 1
    return False



In [116]:
### References --> https://towardsdatascience.com/selenium-tweepy-to-scrap-tweets-from-tweeter-and-analysing-sentiments-1804db3478ac

def get_tweets(query, desired_lang = 'hi',maxTweets = 10): 
        # empty list to store parsed tweets 
        tweets = [] 
        texts_list = []
        sinceId = None
        max_id = -1
        tweetCount = 0
        tweetsPerQry = 10
        
        while tweetCount < maxTweets:
            try:
                if (max_id <= 0):
                    if (not sinceId):
                        new_tweets = api.search(q=query, lang= desired_lang, tweet_mode= 'extended',count=tweetsPerQry)
                    else:
                        new_tweets = api.search(q=query, lang = desired_lang, tweet_mode= 'extended',count=tweetsPerQry,
                                                since_id=sinceId)
                else:
                    if (not sinceId):
                        new_tweets = api.search(q=query, lang = desired_lang, tweet_mode= 'extended', count=tweetsPerQry,
                                                max_id=str(max_id - 1))
                    else:
                        new_tweets = api.search(q=query, lang = desired_lang, tweet_mode= 'extended', count=tweetsPerQry,
                                                max_id=str(max_id - 1),
                                                since_id=sinceId)
                if not new_tweets:
                    print("No more tweets found")
                    break
                    
                for tweet in new_tweets:

                    parsed_tweet = {} 

                    # Making Sure that the tweets don't have more than 50% word similarity

                    already_existing_tweet_flag = check_tweet_exists(tweet.full_text,texts_list)
                    if(already_existing_tweet_flag):
                        continue
                    else:
                        tweets.append(parsed_tweet)
                        texts_list.append(tweet.full_text)


                    parsed_tweet['id']                     = tweet.id
                    parsed_tweet['conversation_id']        = tweet.id_str
                    parsed_tweet['created_at']             = tweet.created_at
                    parsed_tweet['date']                   = str(tweet.created_at).split()[0]
                    parsed_tweet['time']                   = str(tweet.created_at).split()[1]
                    parsed_tweet['time_zone']              = "UTC" 
                    parsed_tweet['user_id']                = tweet.user.id
                    parsed_tweet['username']               = tweet.user.screen_name
                    parsed_tweet['name']                   = tweet.user.name
                    parsed_tweet['place']                  = tweet.user.location
                    parsed_tweet['tweet']                  = tweet.full_text
                    parsed_tweet['mentions']               = get_list_of_user_mentions(tweet)
                    parsed_tweet['urls']                   = tweet._json['entities']['urls']
                    parsed_tweet['photos']                 = get_list_of_photos(tweet)
                    parsed_tweet['replies_count']          = 0 #get_replies_count(tweet,tweet.user.screen_name)
                    parsed_tweet['retweet_count']          = tweet.retweet_count
                    parsed_tweet['likes_count']            = tweet.favorite_count
                    parsed_tweet['hashtags']               = get_all_hashtags(tweet)
                    parsed_tweet['cashtags']               = []
                    parsed_tweet['link']                   = "http://twitter.com/anyuser/status/" + tweet.id_str
                    parsed_tweet['retweet']                = tweet.retweeted
                    parsed_tweet['quote_url']              = " "
                    parsed_tweet['video']                  = 0
                    parsed_tweet['geo']                    = tweet._json['geo']
                    parsed_tweet['source']                 = tweet._json['source']
                    parsed_tweet['rt_user_id']             = get_retweet_user_id(tweet)
                    parsed_tweet['retweet_date']           = get_retweet_date(tweet)
                    parsed_tweet['retweet_id']             = get_retweet_id(tweet)
                                     
                tweetCount += len(new_tweets)
                print("Downloaded {0} tweets".format(tweetCount))
                max_id = new_tweets[-1].id

            except tweepy.TweepError as e:
                print("Tweepy error : " + str(e))
                break
        
        return pd.DataFrame(tweets)

In [117]:
df = get_tweets("covid",'bn')
df

Downloaded 9 tweets
Downloaded 19 tweets


,id,conversation_id,created_at,date,time,time_zone,user_id,username,name,place,tweet,mentions,urls,photos,replies_count,retweet_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,geo,source,rt_user_id,retweet_date,retweet_id
0,1277551535367401473,1277551535367401473,2020-06-29 10:36:36,2020-06-29,10:36:36,UTC,1131532751084052481,UtpalPatowary7,Utpal Patowary,,@himantabiswa বহুত হ’ল Covid show .আৰু নকৰিব ৷...,[himantabiswa],[],[],0,0,0,[],[],http://twitter.com/anyuser/status/127755153536...,False,,0,None,"<a href=""http://twitter.com/download/android"" ...",,,0
1,1277551271956733954,1277551271956733954,2020-06-29 10:35:33,2020-06-29,10:35:33,UTC,795241907597123584,aajkaalofficial,Aajkaal Official,"Kolkata, India",১৪ শতাংশ কলকাতাবাসীই এখন কোভিড প্রতিরোধে সক্ষম...,[],"[{'url': 'https://t.co/cwc1IFavhz', 'expanded_...",[https://pbs.twimg.com/media/EbrGGQvU0AItusS.jpg],0,0,0,[Aajkaal],[],http://twitter.com/anyuser/status/127755127195...,False,,0,None,"<a href=""https://mobile.twitter.com"" rel=""nofo...",,,0
2,1277549778386268165,1277549778386268165,2020-06-29 10:29:37,2020-06-29,10:29:37,UTC,1032488013341777925,Purbali9,Purbali,,RT @News18Northeast: ‘মই উশাহ ল’ব পৰা নাই দেউত...,[News18Northeast],"[{'url': 'https://t.co/p7t3r7XbNQ', 'expanded_...",[],0,2,0,[],[],http://twitter.com/anyuser/status/127754977838...,False,,0,None,"<a href=""http://twitter.com/download/android"" ...",739053070932287488,Mon Jun 29 05:33:35 +0000 2020,1277475278688628736
3,1277549463003754498,1277549463003754498,2020-06-29 10:28:22,2020-06-29,10:28:22,UTC,713644569464705025,SangbadPratidin,Sangbad Pratidin,Kolkata,ওই ব্যক্তির করোনা পরীক্ষার রিপোর্ট এখনও হাতে আ...,[],"[{'url': 'https://t.co/pSv2dElq7V', 'expanded_...",[],0,0,3,"[CoronaUpdatesInIndia, SangbadPratidin]",[],http://twitter.com/anyuser/status/127754946300...,False,,0,None,"<a href=""https://mobile.twitter.com"" rel=""nofo...",,,0
4,1277548851516194818,1277548851516194818,2020-06-29 10:25:56,2020-06-29,10:25:56,UTC,1245654032867143684,DekaBarasha9,Barasha Deka,,RT @Bihuboliya_moi: গুৱাহাটী মহানগৰীত ভয়ংকৰ ৰ...,[Bihuboliya_moi],"[{'url': 'https://t.co/8p0DmBLcVQ', 'expanded_...",[],0,1,0,[],[],http://twitter.com/anyuser/status/127754885151...,False,,0,None,"<a href=""http://twitter.com/download/android"" ...",1222482839213494272,Mon Jun 29 03:54:48 +0000 2020,1277450420172939264
5,1277548102237462529,1277548102237462529,2020-06-29 10:22:57,2020-06-29,10:22:57,UTC,1121758041731260416,SumanMa05693443,Suman Mahapatra,"ঝাড়গ্রাম,নয়াগ্রাম",RT @Asishsahoo12341: নয়াগ্রাম ব্লক তৃণমূল কংগ...,[Asishsahoo12341],[],[],0,17,0,[],[],http://twitter.com/anyuser/status/127754810223...,False,,0,None,"<a href=""http://twitter.com/download/android"" ...",1255000189439545344,Sun Jun 28 16:08:49 +0000 2020,1277272752676732928
6,1277544765538447360,1277544765538447360,2020-06-29 10:09:42,2020-06-29,10:09:42,UTC,401472150,DesheBideshe,Deshe Bideshe,Toronto,করোনা পরীক্ষার ফি নির্ধারণ করে পরিপত্র\nhttps:...,[],"[{'url': 'https://t.co/KVpAgwkGwp', 'expanded_...",[],0,0,0,"[CoronavirusPandemic, Covid_19]",[],http://twitter.com/anyuser/status/127754476553...,False,,0,None,"<a href=""https://mobile.twitter.com"" rel=""nofo...",,,0
7,1277544295499587584,1277544295499587584,2020-06-29 10:07:50,2020-06-29,10:07:50,UTC,1262775128531914753,HritankarC,Hritankar Chakraborty,,"RT @PIBAgartala: #COVID_19 ২৯ জুন, ২০২০ সকাল ৮...",[PIBAgartala],[],[],0,35,0,"[COVID_19, CoronawatchIndia, IndiaFightsCorona]",[],http://twitter.com/anyuser/status/127754429549...,False,,0,None,"<a href=""http://twitter.com/download/android"" ...",902067067104354306,Mon Jun 29 06:02:58 +0000 2020,1277482672231608321
8,1277541580270792705,1277541580270792705,2020-06-29 09:57:02,2020-06-29,09:57:02,UTC,1243866162821083136,mustafa52953906,mustafa,"Patna, India",RT @BanglarGorboMB: নদিয়ার কল্যাণীর কলেজ অফ ম...,[BanglarGorboMB],[],[],0,142,0,[],[],http://twitter.com/anyuser/status/127754158027...,False,,0,None,"<a href=""http://twitter.com/download/an

In [118]:
df.to_csv("/gdrive/My Drive/2nd Sem/IBM/Covid Project/Dataset/Scraping Tweets Demo/covid_tweets_raw_features_dump.csv",index=False)

#Extracting User Features

## User Features

![alt text](https://drive.google.com/uc?id=1iXpdyY9eQw-HN7knBKdfhOhf4SPDqPlr)

In [119]:
df = pd.read_csv("/gdrive/My Drive/2nd Sem/IBM/Covid Project/Dataset/Scraping Tweets Demo/covid_tweets_raw_features_dump.csv")
df

,id,conversation_id,created_at,date,time,time_zone,user_id,username,name,place,tweet,mentions,urls,photos,replies_count,retweet_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,geo,source,rt_user_id,retweet_date,retweet_id
0,1277551535367401473,1277551535367401473,2020-06-29 10:36:36,2020-06-29,10:36:36,UTC,1131532751084052481,UtpalPatowary7,Utpal Patowary,NaN,@himantabiswa বহুত হ’ল Covid show .আৰু নকৰিব ৷...,['himantabiswa'],[],[],0,0,0,[],[],http://twitter.com/anyuser/status/127755153536...,False,,0,NaN,"<a href=""http://twitter.com/download/android"" ...",NaN,NaN,0
1,1277551271956733954,1277551271956733954,2020-06-29 10:35:33,2020-06-29,10:35:33,UTC,795241907597123584,aajkaalofficial,Aajkaal Official,"Kolkata, India",১৪ শতাংশ কলকাতাবাসীই এখন কোভিড প্রতিরোধে সক্ষম...,[],"[{'url': 'https://t.co/cwc1IFavhz', 'expanded_...",['https://pbs.twimg.com/media/EbrGGQvU0AItusS....,0,0,0,['Aajkaal'],[],http://twitter.com/anyuser/status/127755127195...,False,,0,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,NaN,0
2,1277549778386268165,1277549778386268165,2020-06-29 10:29:37,2020-06-29,10:29:37,UTC,1032488013341777925,Purbali9,Purbali,NaN,RT @News18Northeast: ‘মই উশাহ ল’ব পৰা নাই দেউত...,['News18Northeast'],"[{'url': 'https://t.co/p7t3r7XbNQ', 'expanded_...",[],0,2,0,[],[],http://twitter.com/anyuser/status/127754977838...,False,,0,NaN,"<a href=""http://twitter.com/download/android"" ...",7.390531e+17,Mon Jun 29 05:33:35 +0000 2020,1277475278688628736
3,1277549463003754498,1277549463003754498,2020-06-29 10:28:22,2020-06-29,10:28:22,UTC,713644569464705025,SangbadPratidin,Sangbad Pratidin,Kolkata,ওই ব্যক্তির করোনা পরীক্ষার রিপোর্ট এখনও হাতে আ...,[],"[{'url': 'https://t.co/pSv2dElq7V', 'expanded_...",[],0,0,3,"['CoronaUpdatesInIndia', 'SangbadPratidin']",[],http://twitter.com/anyuser/status/127754946300...,False,,0,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,NaN,0
4,1277548851516194818,1277548851516194818,2020-06-29 10:25:56,2020-06-29,10:25:56,UTC,1245654032867143684,DekaBarasha9,Barasha Deka,NaN,RT @Bihuboliya_moi: গুৱাহাটী মহানগৰীত ভয়ংকৰ ৰ...,['Bihuboliya_moi'],"[{'url': 'https://t.co/8p0DmBLcVQ', 'expanded_...",[],0,1,0,[],[],http://twitter.com/anyuser/status/127754885151...,False,,0,NaN,"<a href=""http://twitter.com/download/android"" ...",1.222483e+18,Mon Jun 29 03:54:48 +0000 2020,1277450420172939264
5,1277548102237462529,1277548102237462529,2020-06-29 10:22:57,2020-06-29,10:22:57,UTC,1121758041731260416,SumanMa05693443,Suman Mahapatra,"ঝাড়গ্রাম,নয়াগ্রাম",RT @Asishsahoo12341: নয়াগ্রাম ব্লক তৃণমূল কংগ...,['Asishsahoo12341'],[],[],0,17,0,[],[],http://twitter.com/anyuser/status/127754810223...,False,,0,NaN,"<a href=""http://twitter.com/download/android"" ...",1.255000e+18,Sun Jun 28 16:08:49 +0000 2020,1277272752676732928
6,1277544765538447360,1277544765538447360,2020-06-29 10:09:42,2020-06-29,10:09:42,UTC,401472150,DesheBideshe,Deshe Bideshe,Toronto,করোনা পরীক্ষার ফি নির্ধারণ করে পরিপত্র\nhttps:...,[],"[{'url': 'https://t.co/KVpAgwkGwp', 'expanded_...",[],0,0,0,"['CoronavirusPandemic', 'Covid_19']",[],http://twitter.com/anyuser/status/127754476553...,False,,0,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,NaN,0
7,1277544295499587584,1277544295499587584,2020-06-29 10:07:50,2020-06-29,10:07:50,UTC,1262775128531914753,HritankarC,Hritankar Chakraborty,NaN,"RT @PIBAgartala: #COVID_19 ২৯ জুন, ২০২০ সকাল ৮...",['PIBAgartala'],[],[],0,35,0,"['COVID_19', 'CoronawatchIndia', 'IndiaFightsC...",[],http://twitter.com/anyuser/status/127754429549...,False,,0,NaN,"<a href=""http://twitter.com/download/android"" ...",9.020671e+17,Mon Jun 29 06:02:58 +0000 2020,1277482672231608321
8,1277541580270792705,1277541580270792705,2020-06-29 09:57:02,2020-06-29,09:57:02,UTC,1243866162821083136,mustafa52953906,mustafa,"Patna, India",RT @BanglarGorboMB: নদিয়ার কল্যাণীর কলেজ অফ ম...,['BanglarGorboMB'],[],[],0,142,0,[],[],http://twitter.com/anyuser/status/127754158027...,False,,0,NaN,"<a href=""http:

In [120]:
def calculate_days_past(tweet_time):
    #Calculating Dates
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    a1 = int(tweet_time[-1])
    b1 = int(months.index(tweet_time[1])+1)
    c1 = int(tweet_time[2])

    #Todays date
    today = date.today()
    life  =  str(date(today.year,today.month,today.day) - date(a1,b1,c1))
    if(len(life)>7):
        life  = life.split(" ")
        life  = life[0]
        life  = int(life)
        return life
    else:
        return 0

In [121]:
def get_user_info(tweet):
    
    tweet_text   = tweet.full_text
    real_name    = tweet._json['user']['name']
    user_handle  = tweet._json['user']['screen_name']
    tweet_id     = tweet._json['id']
    user_id      = tweet._json['user']['id']
    
    user_info = api.get_user(user_id,user_mode='extended')
    desc = user_info._json['description']
    
    try:
        expanded_url = user_info._json['entities']['url']['urls'][0]['expanded_url']
        expanded_url = 1
    except:
        expanded_url = 0

    
    tweet_link             = "http://twitter.com/anyuser/status/" + str(tweet.id)
    chars_in_desc          = len(desc)
    chars_in_real_name     = len(real_name)
    chars_in_user_handle   = len(user_handle)
    num_matches            = len(difflib.SequenceMatcher(None, real_name,user_handle).get_matching_blocks())
    total_urls_in_desc     = len(user_info._json['entities']['description']['urls'])
    official_url_exists    = expanded_url
    followers_count        = user_info._json['followers_count']
    friends_count          = user_info._json['friends_count']
    listed_count           = user_info._json['listed_count']
    favourites_count       = user_info._json['favourites_count']
    geo_enabled            = int(user_info._json['geo_enabled'])
    acc_created_on         = user_info._json['created_at'].split()
    acc_life               = calculate_days_past(acc_created_on)
    verified               = int(user_info._json['verified'])
    num_tweets             = user_info._json['statuses_count']
    protected              = int(user_info._json['protected'])
    avg_likes_per_tweet    = favourites_count/num_tweets
    latest_tweet_time      = user_info._json['status']['created_at'].split()
    activity               = calculate_days_past(latest_tweet_time)

    if(acc_life==0):
        posting_frequency = num_tweets
    else:
        posting_frequency      = num_tweets/acc_life

    
    if(friends_count!=0):
        follower_friends_ratio = followers_count/friends_count
    else:
        follower_friends_ratio = 9999999
    
    list = [tweet_text, tweet_link, desc,
            real_name, user_handle, user_id,
            chars_in_desc, chars_in_real_name,
            chars_in_user_handle, num_matches,
            total_urls_in_desc, official_url_exists,
            followers_count, friends_count,
            listed_count, favourites_count,
            geo_enabled, acc_created_on,
            acc_life, verified, num_tweets,
            protected, posting_frequency,
            latest_tweet_time, activity,
            avg_likes_per_tweet, follower_friends_ratio]
    
    # for item in list:
    #     print(item)
        
    return list

In [122]:
tweet_ids_list = df['id'].values.tolist()

In [123]:
# Creating User Features Dataset
count = 1
new_users_features_list = []

for id in range(len(tweet_ids_list)):


    try:

        tweet = api.get_status(df['id'][id],tweet_mode='extended')
        user_info = get_user_info(tweet)
        user_info.insert(0,df['id'][id])
        new_users_features_list.append(user_info)
        count = count+1

        if(count>300):
            break

    except tweepy.TweepError as e:
        print("Tweepy error : " + str(e))
        continue

print(len(new_users_features_list))

13


In [124]:
print(len(new_users_features_list))
print(count)

13
14


In [125]:
df = pd.DataFrame(new_users_features_list,columns = [ 'tweet_id', 'tweet_text', 'tweet_link', 'desc', 'real_name', 'user_handle', 'user_id', 'chars_in_desc', 'chars_in_real_name',
                                                            'chars_in_user_handle', 'num_matches', 'total_urls_in_desc', 'official_url_exists', 'followers_count',
                                                            'friends_count', 'listed_count', 'favourites_count', 'geo_enabled', 'acc_created_on', 'acc_life', 'verified',
                                                            'num_tweets', 'protected', 'posting_frequency', 'latest_tweet_time', 'activity', 'avg_likes_per_tweet',
                                                            'follower_friends_ratio'])


In [126]:
df

,tweet_id,tweet_text,tweet_link,desc,real_name,user_handle,user_id,chars_in_desc,chars_in_real_name,chars_in_user_handle,num_matches,total_urls_in_desc,official_url_exists,followers_count,friends_count,listed_count,favourites_count,geo_enabled,acc_created_on,acc_life,verified,num_tweets,protected,posting_frequency,latest_tweet_time,activity,avg_likes_per_tweet,follower_friends_ratio
0,1277551535367401473,@himantabiswa বহুত হ’ল Covid show .আৰু নকৰিব ৷...,http://twitter.com/anyuser/status/127755153536...,hardworking makes a man perfect,Utpal Patowary,UtpalPatowary7,1131532751084052481,31,14,14,3,0,0,0,25,0,47,0,"[Thu, May, 23, 12:10:05, +0000, 2019]",403,0,14,0,0.034739,"[Mon, Jun, 29, 10:47:18, +0000, 2020]",0,3.357143,0.000000
1,1277551271956733954,১৪ শতাংশ কলকাতাবাসীই এখন কোভিড প্রতিরোধে সক্ষম...,http://twitter.com/anyuser/status/127755127195...,Facebook: https://t.co/vQFs4apIJo\nYoutube Cha...,Aajkaal Official,aajkaalofficial,795241907597123584,111,16,15,3,3,1,354,134,1,11,0,"[Sun, Nov, 06, 12:30:20, +0000, 2016]",1331,0,4134,0,3.105935,"[Mon, Jun, 29, 10:35:33, +0000, 2020]",0,0.002661,2.641791
2,1277549778386268165,RT @News18Northeast: ‘মই উশাহ ল’ব পৰা নাই দেউত...,http://twitter.com/anyuser/status/127754977838...,,Purbali,Purbali9,1032488013341777925,0,7,8,2,0,0,117,205,0,39,0,"[Thu, Aug, 23, 04:41:39, +0000, 2018]",676,0,89,0,0.131657,"[Mon, Jun, 29, 10:30:08, +0000, 2020]",0,0.438202,0.570732
3,1277549463003754498,ওই ব্যক্তির করোনা পরীক্ষার রিপোর্ট এখনও হাতে আ...,http://twitter.com/anyuser/status/127754946300...,"Official Twitter account of Sangbad Pratidin, ...",Sangbad Pratidin,SangbadPratidin,713644569464705025,124,16,15,3,0,1,20818,216,168,109,1,"[Sat, Mar, 26, 08:31:39, +0000, 2016]",1556,0,46998,0,30.204370,"[Mon, Jun, 29, 10:59:15, +0000, 2020]",0,0.002319,96.379630
4,1277548851516194818,RT @Bihuboliya_moi: গুৱাহাটী মহানগৰীত ভয়ংকৰ ৰ...,http://twitter.com/anyuser/status/127754885151...,If our journey is same then why don’t you foll...,Barasha Deka,DekaBarasha9,1245654032867143684,67,12,12,2,0,0,601,483,0,599,0,"[Thu, Apr, 02, 10:07:32, +0000, 2020]",88,0,307,0,3.488636,"[Mon, Jun, 29, 10:25:56, +0000, 2020]",0,1.951140,1.244306
5,1277548102237462529,RT @Asishsahoo12341: নয়াগ্রাম ব্লক তৃণমূল কংগ...,http://twitter.com/anyuser/status/127754810223...,বাঙ্গলার জয় \nমা মাটি মানুষের জয়,Suman Mahapatra,SumanMa05693443,1121758041731260416,31,15,15,3,0,0,27,45,0,44,0,"[Fri, Apr, 26, 12:48:53, +0000, 2019]",430,0,11,0,0.025581,"[Mon, Jun, 29, 10:22:59, +0000, 2020]",0,4.000000,0.600000
6,1277544765538447360,করোনা পরীক্ষার ফি নির্ধারণ করে পরিপত্র\nhttps:...,http://twitter.com/anyuser/status/127754476553...,"An interactive, new-generation multimedia news...",Deshe Bideshe,DesheBideshe,401472150,160,13,12,3,0,1,12127,4,21,0,0,"[Sun, Oct, 30, 16:03:38, +0000, 2011]",3165,0,199298,0,62.969352,"[Mon, Jun, 29, 11:03:56, +0000, 2020]",0,0.000000,3031.750000
7,1277544295499587584,"RT @PIBAgartala: #COVID_19 ২৯ জুন, ২০২০ সকাল ৮...",http://twitter.com/anyuser/status/127754429549...,lam Hritankar Chakraborty . student of Diet ka...,Hritankar Chakraborty,HritankarC,1262775128531914753,78,21,10,3,0,0,26,105,0,1020,0,"[Tue, May, 19, 16:00:46, +0000, 2020]",41,0,1044,0,25.463415,"[Mon, Jun, 29, 10:08:01, +0000, 2020]",0,0.977011,0.247619
8,1277541580270792705,RT @BanglarGorboMB: নদিয়ার কল্যাণীর কলেজ অফ ম...,http://twitter.com/anyuser/status/127754158027...,Not_Political,mustafa,mustafa52953906,1243866162821083136,13,7,15,2,0,0,33,288,0,1707,1,"[Sat, Mar, 28, 11:43:10, +0000, 2020]",93,0,2080,0,22.365591,"[Mon, Jun, 29, 10:03:36, +0000, 2020]",0,0.820673,0.114583
9,1277541524570439680,এই টিকা ১০০% করোনা-রোধী সক্রিয় অ্যান্টিবডি তৈ...,http://twitter.com/anyuser/status/127754152457...,West Bengal News 24 is an interactive Bengali ...,WestBengal News 24,WBNews_24,1135787487652003840,136,18,9,5,0,1,876,32,0,15,0,"[Tue, Jun, 04, 05:56:53, +0000, 2019]",391,0,7811,0,19.976982,"[Mon, Jun, 29, 10:47:20, +0000, 2020]",0,0.001920,27.375000


In [127]:
def preprocess_tweet(tweet):

    tweet = tweet.lower()
    url = r'http\S+'
    tweet = re.sub(url, 'URL', tweet, flags=re.MULTILINE)
    emoji = re.compile("["         u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   u"\U00002500-\U00002BEF"  # chinese char
                                   u"\U00002702-\U000027B0"
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   u"\U0001f926-\U0001f937"
                                   u"\U00010000-\U0010ffff"
                                   u"\u2640-\u2642"
                                   u"\u2600-\u2B55"
                                   u"\u200d"
                                   u"\u23cf"
                                   u"\u23e9"
                                   u"\u231a"
                                   u"\ufe0f"  # dingbats
                                   u"\u3030"
                                   "]+", flags=re.UNICODE)
    tweet =  emoji.sub(r'', tweet)
    tweet = ' '.join([word[1:] if word[0] == '#' else word for word in tweet.split()])
    
    return tweet

In [128]:
pip install googletrans

In [129]:
from googletrans import Translator
translator = Translator()

In [130]:
import re

for i in range(len(df)):
    df.at[i,'tweet_text'] = preprocess_tweet(df.iloc[i]['tweet_text'])

In [131]:
def too_much_cleaned_tweet(tweet):

    
    cleaned_text = tweet.split()
    exclude_char_list = ['#','@','rt','|','%', '[', '(', '{', '}', ']', ')','url','URL','covid','covid19','corona','coronavirus','0','1','2','3','4','5','6','7','8','9']
    tweet_without_special_chars = [word for word in cleaned_text if all(ch not in word for ch in exclude_char_list)]
    tweet_without_special_chars = ' '.join(tweet_without_special_chars)
 
    return tweet_without_special_chars

    


In [132]:
list = []
for tweet_text in df['tweet_text']:
    list.append(too_much_cleaned_tweet(tweet_text))

cleaned_tweet_df = pd.DataFrame(list,columns = ['too_much_cleaned_tweet'])
cleaned_tweet_df

,too_much_cleaned_tweet
0,বহুত হ’ল show .আৰু নকৰিব ৷ ৰাইজক আমুৱাইছে ৷ pr...
1,১৪ শতাংশ কলকাতাবাসীই এখন কোভিড প্রতিরোধে সক্ষম...
2,"‘মই উশাহ ল’ব পৰা নাই দেউতা’, মৃত্যুৰ পূৰ্বে ছে..."
3,ওই ব্যক্তির করোনা পরীক্ষার রিপোর্ট এখনও হাতে আ...
4,"গুৱাহাটী মহানগৰীত ভয়ংকৰ ৰূপ কোৰোণাৰ, বিগত ২৪ ..."
5,নয়াগ্রাম ব্লক তৃণমূল কংগ্রেসের সম্মানীয় সভাপ...
6,করোনা পরীক্ষার ফি নির্ধারণ করে পরিপত্র
7,"২৯ জুন, ২০২০ সকাল ৮. টা পর্যন্ত ত্রিপুরায় কোভ..."
8,নদিয়ার কল্যাণীর কলেজ অফ মেডিসিন ও জেএনএম হাসপ...
9,এই টিকা করোনা-রোধী সক্রিয় অ্যান্টিবডি তৈরিতে ...


In [133]:
df = pd.concat([df,cleaned_tweet_df],axis=1)
df

,tweet_id,tweet_text,tweet_link,desc,real_name,user_handle,user_id,chars_in_desc,chars_in_real_name,chars_in_user_handle,num_matches,total_urls_in_desc,official_url_exists,followers_count,friends_count,listed_count,favourites_count,geo_enabled,acc_created_on,acc_life,verified,num_tweets,protected,posting_frequency,latest_tweet_time,activity,avg_likes_per_tweet,follower_friends_ratio,too_much_cleaned_tweet
0,1277551535367401473,@himantabiswa বহুত হ’ল covid show .আৰু নকৰিব ৷...,http://twitter.com/anyuser/status/127755153536...,hardworking makes a man perfect,Utpal Patowary,UtpalPatowary7,1131532751084052481,31,14,14,3,0,0,0,25,0,47,0,"[Thu, May, 23, 12:10:05, +0000, 2019]",403,0,14,0,0.034739,"[Mon, Jun, 29, 10:47:18, +0000, 2020]",0,3.357143,0.000000,বহুত হ’ল show .আৰু নকৰিব ৷ ৰাইজক আমুৱাইছে ৷ pr...
1,1277551271956733954,১৪ শতাংশ কলকাতাবাসীই এখন কোভিড প্রতিরোধে সক্ষম...,http://twitter.com/anyuser/status/127755127195...,Facebook: https://t.co/vQFs4apIJo\nYoutube Cha...,Aajkaal Official,aajkaalofficial,795241907597123584,111,16,15,3,3,1,354,134,1,11,0,"[Sun, Nov, 06, 12:30:20, +0000, 2016]",1331,0,4134,0,3.105935,"[Mon, Jun, 29, 10:35:33, +0000, 2020]",0,0.002661,2.641791,১৪ শতাংশ কলকাতাবাসীই এখন কোভিড প্রতিরোধে সক্ষম...
2,1277549778386268165,rt @news18northeast: ‘মই উশাহ ল’ব পৰা নাই দেউত...,http://twitter.com/anyuser/status/127754977838...,,Purbali,Purbali9,1032488013341777925,0,7,8,2,0,0,117,205,0,39,0,"[Thu, Aug, 23, 04:41:39, +0000, 2018]",676,0,89,0,0.131657,"[Mon, Jun, 29, 10:30:08, +0000, 2020]",0,0.438202,0.570732,"‘মই উশাহ ল’ব পৰা নাই দেউতা’, মৃত্যুৰ পূৰ্বে ছে..."
3,1277549463003754498,ওই ব্যক্তির করোনা পরীক্ষার রিপোর্ট এখনও হাতে আ...,http://twitter.com/anyuser/status/127754946300...,"Official Twitter account of Sangbad Pratidin, ...",Sangbad Pratidin,SangbadPratidin,713644569464705025,124,16,15,3,0,1,20818,216,168,109,1,"[Sat, Mar, 26, 08:31:39, +0000, 2016]",1556,0,46998,0,30.204370,"[Mon, Jun, 29, 10:59:15, +0000, 2020]",0,0.002319,96.379630,ওই ব্যক্তির করোনা পরীক্ষার রিপোর্ট এখনও হাতে আ...
4,1277548851516194818,rt @bihuboliya_moi: গুৱাহাটী মহানগৰীত ভয়ংকৰ ৰ...,http://twitter.com/anyuser/status/127754885151...,If our journey is same then why don’t you foll...,Barasha Deka,DekaBarasha9,1245654032867143684,67,12,12,2,0,0,601,483,0,599,0,"[Thu, Apr, 02, 10:07:32, +0000, 2020]",88,0,307,0,3.488636,"[Mon, Jun, 29, 10:25:56, +0000, 2020]",0,1.951140,1.244306,"গুৱাহাটী মহানগৰীত ভয়ংকৰ ৰূপ কোৰোণাৰ, বিগত ২৪ ..."
5,1277548102237462529,rt @asishsahoo12341: নয়াগ্রাম ব্লক তৃণমূল কংগ...,http://twitter.com/anyuser/status/127754810223...,বাঙ্গলার জয় \nমা মাটি মানুষের জয়,Suman Mahapatra,SumanMa05693443,1121758041731260416,31,15,15,3,0,0,27,45,0,44,0,"[Fri, Apr, 26, 12:48:53, +0000, 2019]",430,0,11,0,0.025581,"[Mon, Jun, 29, 10:22:59, +0000, 2020]",0,4.000000,0.600000,নয়াগ্রাম ব্লক তৃণমূল কংগ্রেসের সম্মানীয় সভাপ...
6,1277544765538447360,করোনা পরীক্ষার ফি নির্ধারণ করে পরিপত্র URL cor...,http://twitter.com/anyuser/status/127754476553...,"An interactive, new-generation multimedia news...",Deshe Bideshe,DesheBideshe,401472150,160,13,12,3,0,1,12127,4,21,0,0,"[Sun, Oct, 30, 16:03:38, +0000, 2011]",3165,0,199298,0,62.969352,"[Mon, Jun, 29, 11:03:56, +0000, 2020]",0,0.000000,3031.750000,করোনা পরীক্ষার ফি নির্ধারণ করে পরিপত্র
7,1277544295499587584,"rt @pibagartala: covid_19 ২৯ জুন, ২০২০ সকাল ৮....",http://twitter.com/anyuser/status/127754429549...,lam Hritankar Chakraborty . student of Diet ka...,Hritankar Chakraborty,HritankarC,1262775128531914753,78,21,10,3,0,0,26,105,0,1020,0,"[Tue, May, 19, 16:00:46, +0000, 2020]",41,0,1044,0,25.463415,"[Mon, Jun, 29, 10:08:01, +0000, 2020]",0,0.977011,0.247619,"২৯ জুন, ২০২০ সকাল ৮. টা পর্যন্ত ত্রিপুরায় কোভ..."
8,1277541580270792705,rt @banglargorbomb: নদিয়ার কল্যাণীর কলেজ অফ ম...,http://twitter.com/anyuser/status/127754158027...,Not_Political,mustafa,mustafa52953906,1243866162821083136,13,7,15,2,0,0,33,288,0,1707,1,"[Sat, Mar, 28, 11:43:10, +0000, 2020]",93,0,2080,0,22.365591,"[Mon, Jun, 29, 10:03:36,

In [134]:
idx_drop = [index for index, row in df.iterrows() if translator.detect(row['too_much_cleaned_tweet']).lang == 'en']
idx_drop

[]

In [135]:
df_modified = copy.deepcopy(df)
df_modified = df_modified.drop(idx_drop)
df_modified = df_modified.drop(['too_much_cleaned_tweet'],axis=1)
df_modified

,tweet_id,tweet_text,tweet_link,desc,real_name,user_handle,user_id,chars_in_desc,chars_in_real_name,chars_in_user_handle,num_matches,total_urls_in_desc,official_url_exists,followers_count,friends_count,listed_count,favourites_count,geo_enabled,acc_created_on,acc_life,verified,num_tweets,protected,posting_frequency,latest_tweet_time,activity,avg_likes_per_tweet,follower_friends_ratio
0,1277551535367401473,@himantabiswa বহুত হ’ল covid show .আৰু নকৰিব ৷...,http://twitter.com/anyuser/status/127755153536...,hardworking makes a man perfect,Utpal Patowary,UtpalPatowary7,1131532751084052481,31,14,14,3,0,0,0,25,0,47,0,"[Thu, May, 23, 12:10:05, +0000, 2019]",403,0,14,0,0.034739,"[Mon, Jun, 29, 10:47:18, +0000, 2020]",0,3.357143,0.000000
1,1277551271956733954,১৪ শতাংশ কলকাতাবাসীই এখন কোভিড প্রতিরোধে সক্ষম...,http://twitter.com/anyuser/status/127755127195...,Facebook: https://t.co/vQFs4apIJo\nYoutube Cha...,Aajkaal Official,aajkaalofficial,795241907597123584,111,16,15,3,3,1,354,134,1,11,0,"[Sun, Nov, 06, 12:30:20, +0000, 2016]",1331,0,4134,0,3.105935,"[Mon, Jun, 29, 10:35:33, +0000, 2020]",0,0.002661,2.641791
2,1277549778386268165,rt @news18northeast: ‘মই উশাহ ল’ব পৰা নাই দেউত...,http://twitter.com/anyuser/status/127754977838...,,Purbali,Purbali9,1032488013341777925,0,7,8,2,0,0,117,205,0,39,0,"[Thu, Aug, 23, 04:41:39, +0000, 2018]",676,0,89,0,0.131657,"[Mon, Jun, 29, 10:30:08, +0000, 2020]",0,0.438202,0.570732
3,1277549463003754498,ওই ব্যক্তির করোনা পরীক্ষার রিপোর্ট এখনও হাতে আ...,http://twitter.com/anyuser/status/127754946300...,"Official Twitter account of Sangbad Pratidin, ...",Sangbad Pratidin,SangbadPratidin,713644569464705025,124,16,15,3,0,1,20818,216,168,109,1,"[Sat, Mar, 26, 08:31:39, +0000, 2016]",1556,0,46998,0,30.204370,"[Mon, Jun, 29, 10:59:15, +0000, 2020]",0,0.002319,96.379630
4,1277548851516194818,rt @bihuboliya_moi: গুৱাহাটী মহানগৰীত ভয়ংকৰ ৰ...,http://twitter.com/anyuser/status/127754885151...,If our journey is same then why don’t you foll...,Barasha Deka,DekaBarasha9,1245654032867143684,67,12,12,2,0,0,601,483,0,599,0,"[Thu, Apr, 02, 10:07:32, +0000, 2020]",88,0,307,0,3.488636,"[Mon, Jun, 29, 10:25:56, +0000, 2020]",0,1.951140,1.244306
5,1277548102237462529,rt @asishsahoo12341: নয়াগ্রাম ব্লক তৃণমূল কংগ...,http://twitter.com/anyuser/status/127754810223...,বাঙ্গলার জয় \nমা মাটি মানুষের জয়,Suman Mahapatra,SumanMa05693443,1121758041731260416,31,15,15,3,0,0,27,45,0,44,0,"[Fri, Apr, 26, 12:48:53, +0000, 2019]",430,0,11,0,0.025581,"[Mon, Jun, 29, 10:22:59, +0000, 2020]",0,4.000000,0.600000
6,1277544765538447360,করোনা পরীক্ষার ফি নির্ধারণ করে পরিপত্র URL cor...,http://twitter.com/anyuser/status/127754476553...,"An interactive, new-generation multimedia news...",Deshe Bideshe,DesheBideshe,401472150,160,13,12,3,0,1,12127,4,21,0,0,"[Sun, Oct, 30, 16:03:38, +0000, 2011]",3165,0,199298,0,62.969352,"[Mon, Jun, 29, 11:03:56, +0000, 2020]",0,0.000000,3031.750000
7,1277544295499587584,"rt @pibagartala: covid_19 ২৯ জুন, ২০২০ সকাল ৮....",http://twitter.com/anyuser/status/127754429549...,lam Hritankar Chakraborty . student of Diet ka...,Hritankar Chakraborty,HritankarC,1262775128531914753,78,21,10,3,0,0,26,105,0,1020,0,"[Tue, May, 19, 16:00:46, +0000, 2020]",41,0,1044,0,25.463415,"[Mon, Jun, 29, 10:08:01, +0000, 2020]",0,0.977011,0.247619
8,1277541580270792705,rt @banglargorbomb: নদিয়ার কল্যাণীর কলেজ অফ ম...,http://twitter.com/anyuser/status/127754158027...,Not_Political,mustafa,mustafa52953906,1243866162821083136,13,7,15,2,0,0,33,288,0,1707,1,"[Sat, Mar, 28, 11:43:10, +0000, 2020]",93,0,2080,0,22.365591,"[Mon, Jun, 29, 10:03:36, +0000, 2020]",0,0.820673,0.114583
9,1277541524570439680,এই টিকা ১০০% করোনা-রোধী সক্রিয় অ্যান্টিবডি তৈ...,http://twitter.com/anyuser/status/127754152457...,West Bengal News 24 is an interactive Bengali ...,WestBengal News 24,WBNews_24,1135787487652003840,136,18,9,5,0,1,876,32,0,15,0,"[Tue, Jun, 04, 05:56:53, +0000, 2019]",391,0,7811,0,19.976982,"[Mon, Jun, 29, 10:47:20, +0000, 2020]",0,0.001920,27.375000


In [136]:
### Adding Label's field --> (( 1=Real || 2=Fake || 3=Maybe ))

df['label'] = pd.DataFrame((np.ones(len(df),dtype=int)*3))
df

,tweet_id,tweet_text,tweet_link,desc,real_name,user_handle,user_id,chars_in_desc,chars_in_real_name,chars_in_user_handle,num_matches,total_urls_in_desc,official_url_exists,followers_count,friends_count,listed_count,favourites_count,geo_enabled,acc_created_on,acc_life,verified,num_tweets,protected,posting_frequency,latest_tweet_time,activity,avg_likes_per_tweet,follower_friends_ratio,too_much_cleaned_tweet,label
0,1277551535367401473,@himantabiswa বহুত হ’ল covid show .আৰু নকৰিব ৷...,http://twitter.com/anyuser/status/127755153536...,hardworking makes a man perfect,Utpal Patowary,UtpalPatowary7,1131532751084052481,31,14,14,3,0,0,0,25,0,47,0,"[Thu, May, 23, 12:10:05, +0000, 2019]",403,0,14,0,0.034739,"[Mon, Jun, 29, 10:47:18, +0000, 2020]",0,3.357143,0.000000,বহুত হ’ল show .আৰু নকৰিব ৷ ৰাইজক আমুৱাইছে ৷ pr...,3
1,1277551271956733954,১৪ শতাংশ কলকাতাবাসীই এখন কোভিড প্রতিরোধে সক্ষম...,http://twitter.com/anyuser/status/127755127195...,Facebook: https://t.co/vQFs4apIJo\nYoutube Cha...,Aajkaal Official,aajkaalofficial,795241907597123584,111,16,15,3,3,1,354,134,1,11,0,"[Sun, Nov, 06, 12:30:20, +0000, 2016]",1331,0,4134,0,3.105935,"[Mon, Jun, 29, 10:35:33, +0000, 2020]",0,0.002661,2.641791,১৪ শতাংশ কলকাতাবাসীই এখন কোভিড প্রতিরোধে সক্ষম...,3
2,1277549778386268165,rt @news18northeast: ‘মই উশাহ ল’ব পৰা নাই দেউত...,http://twitter.com/anyuser/status/127754977838...,,Purbali,Purbali9,1032488013341777925,0,7,8,2,0,0,117,205,0,39,0,"[Thu, Aug, 23, 04:41:39, +0000, 2018]",676,0,89,0,0.131657,"[Mon, Jun, 29, 10:30:08, +0000, 2020]",0,0.438202,0.570732,"‘মই উশাহ ল’ব পৰা নাই দেউতা’, মৃত্যুৰ পূৰ্বে ছে...",3
3,1277549463003754498,ওই ব্যক্তির করোনা পরীক্ষার রিপোর্ট এখনও হাতে আ...,http://twitter.com/anyuser/status/127754946300...,"Official Twitter account of Sangbad Pratidin, ...",Sangbad Pratidin,SangbadPratidin,713644569464705025,124,16,15,3,0,1,20818,216,168,109,1,"[Sat, Mar, 26, 08:31:39, +0000, 2016]",1556,0,46998,0,30.204370,"[Mon, Jun, 29, 10:59:15, +0000, 2020]",0,0.002319,96.379630,ওই ব্যক্তির করোনা পরীক্ষার রিপোর্ট এখনও হাতে আ...,3
4,1277548851516194818,rt @bihuboliya_moi: গুৱাহাটী মহানগৰীত ভয়ংকৰ ৰ...,http://twitter.com/anyuser/status/127754885151...,If our journey is same then why don’t you foll...,Barasha Deka,DekaBarasha9,1245654032867143684,67,12,12,2,0,0,601,483,0,599,0,"[Thu, Apr, 02, 10:07:32, +0000, 2020]",88,0,307,0,3.488636,"[Mon, Jun, 29, 10:25:56, +0000, 2020]",0,1.951140,1.244306,"গুৱাহাটী মহানগৰীত ভয়ংকৰ ৰূপ কোৰোণাৰ, বিগত ২৪ ...",3
5,1277548102237462529,rt @asishsahoo12341: নয়াগ্রাম ব্লক তৃণমূল কংগ...,http://twitter.com/anyuser/status/127754810223...,বাঙ্গলার জয় \nমা মাটি মানুষের জয়,Suman Mahapatra,SumanMa05693443,1121758041731260416,31,15,15,3,0,0,27,45,0,44,0,"[Fri, Apr, 26, 12:48:53, +0000, 2019]",430,0,11,0,0.025581,"[Mon, Jun, 29, 10:22:59, +0000, 2020]",0,4.000000,0.600000,নয়াগ্রাম ব্লক তৃণমূল কংগ্রেসের সম্মানীয় সভাপ...,3
6,1277544765538447360,করোনা পরীক্ষার ফি নির্ধারণ করে পরিপত্র URL cor...,http://twitter.com/anyuser/status/127754476553...,"An interactive, new-generation multimedia news...",Deshe Bideshe,DesheBideshe,401472150,160,13,12,3,0,1,12127,4,21,0,0,"[Sun, Oct, 30, 16:03:38, +0000, 2011]",3165,0,199298,0,62.969352,"[Mon, Jun, 29, 11:03:56, +0000, 2020]",0,0.000000,3031.750000,করোনা পরীক্ষার ফি নির্ধারণ করে পরিপত্র,3
7,1277544295499587584,"rt @pibagartala: covid_19 ২৯ জুন, ২০২০ সকাল ৮....",http://twitter.com/anyuser/status/127754429549...,lam Hritankar Chakraborty . student of Diet ka...,Hritankar Chakraborty,HritankarC,1262775128531914753,78,21,10,3,0,0,26,105,0,1020,0,"[Tue, May, 19, 16:00:46, +0000, 2020]",41,0,1044,0,25.463415,"[Mon, Jun, 29, 10:08:01, +0000, 2020]",0,0.977011,0.247619,"২৯ জুন, ২০২০ সকাল ৮. টা পর্যন্ত ত্রিপুরায় কোভ...",3
8,1277541580270792705,rt @banglargorbomb: নদিয়ার কল্যাণীর কলেজ অফ ম...,http://twitter.com/anyuser/status/127754158027...,Not_Political,mustafa,mustafa52953906,1243866162821083136,13,7,15,2,0,0,33,288,0,1707,1,"[Sat, Mar, 28, 11:43:10, +0000, 2020]",93,0,2080,0,22.365591,"[M

In [137]:
df.to_csv("/gdrive/My Drive/2nd Sem/IBM/Covid Project/Dataset/Scraping Tweets Demo/covid_tweets_with_user_features.csv",index=False)